# Measuring Single Chip Performance on Tensor Processing Tasks


In [1]:
from pathlib import Path
import sys

from typing import Literal

import jax

platform : Literal["darwin", "colab", "cuda"] = "darwin"

try:
    import google.colab
    platform = "colab"
except ImportError:
    devices = jax.devices()
    if any(d.platform == "gpu" for d in devices):
        platform = "cuda"

print(f"Running on {platform}")

if platform == "colab":
    !git clone https://github.com/novastar53/high_performance_jax
    !cd high_performance_jax && git pull
    !git clone https://github.com/novastar53/deepkit
    !cd deepkit && git pull
    hpj_dir = str(Path().absolute() / "high_performance_jax" / "src" )
    dt_dir = str(Path().absolute() / "deepkit" / "src" )
    sys.path.append(hpj_dir)
    print(hpj_dir)
    sys.path.append(dt_dir)
    print(dt_dir)

Running on colab
fatal: destination path 'high_performance_jax' already exists and is not an empty directory.
Already up to date.
fatal: destination path 'deepkit' already exists and is not an empty directory.
Already up to date.
/content/high_performance_jax/src
/content/deepkit/src


In [7]:
from deepkit.utils import timeit
from high_performance_jax.single_chip_performance import *
import matplotlib.pyplot as plt

dtype = jnp.bfloat16
devices = jax.devices()
print("Devices:")
for i,d in enumerate(devices):
  print(f"{i+1}. {d.device_kind}")    # e.g. “TPU v3”


#####################################
##        jax.lax matmul presets   ##
#####################################
## 'ANY_F8_ANY_F8_F32',
## 'ANY_F8_ANY_F8_F32_FAST_ACCUM'
## 'ANY_F8_ANY_F8_ANY'
## 'ANY_F8_ANY_F8_ANY_FAST_ACCUM'
## 'F16_F16_F16'
## 'F16_F16_F32'
## 'BF16_BF16_BF16'
## 'BF16_BF16_F32'
## 'BF16_BF16_F32_X3'
## 'BF16_BF16_F32_X6'
## 'TF32_TF32_F32'
## 'TF32_TF32_F32_X3'
## 'F32_F32_F32'
## 'F64_F64_F64'
#####################################
jax.config.update("jax_default_matmul_precision", "BF16_BF16_F32") # Set the default precision for matrix multiplication

dims = [2**i for i in range(6, 15)]
times = []
tflops = []
hbm_rates = []
intensities = []

for dim in dims:
    n = 2**29 // dim ** 2
    A = jnp.ones((n, dim, dim), dtype=dtype)
    B = jnp.ones((n, dim, dim), dtype=dtype)
    #C = jnp.ones((dim, dim), dtype=dtype)
    task = "matmul"

    average_time_ms = timeit(jax.jit(bmm), A, B)
    flops = measure_tpu_flops(task, n, dim)
    times.append(average_time_ms)
    tflops.append(1000 * flops / average_time_ms / 10**12)
    hbm_xfer = measure_tpu_hbm_memory_transfer(task, n, dim, dtype)
    hbm_rates.append(1000 * hbm_xfer / average_time_ms)
    arithmetic_intensity = flops / hbm_xfer
    intensities.append(arithmetic_intensity)

    print(f"{n=} | {dim=} | average time (ms): {average_time_ms:.2f} | "
            f"hbm xfer/s {hbm_rates[-1]:.2e} | "
            f"flops {flops:.2e} | "
            f"intensity {arithmetic_intensity:,.4f} | "
            f"teraflops/s {tflops[-1]:,.4f}")

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 5))

ax1.plot(dims, times, marker='o')
#ax1.set_xscale('log')
ax1.set_xlabel("Matrix Dimension (dim)")
ax1.set_ylabel("Average Time (ms)")
ax1.set_title("Matrix Multiplication Time vs Dimension")
ax1.grid(True)

ax2.plot(intensities, tflops, marker='o')
#ax1.set_xscale('log')
ax2.set_xlabel("Arithmetic Intensity")
ax2.set_ylabel("TFLOPS/s")
ax2.set_title("TFLOPS/s vs Artithmetic Intensity")
ax2.grid(True)

ax3.plot(dims, hbm_rates, marker='o')
#ax3.set_xscale('log')
ax3.set_xlabel("Matrix Dimension (dim)")
ax3.set_ylabel("HBM Transfer Rate (bytes/s)")
ax3.set_title("HBM Transfer Rate vs Dimension")
ax3.grid(True)

plt.tight_layout()
plt.show()

Devices:
1. TPU v5 lite
n=131072 | dim=64 | average time (ms): 27.40 | hbm xfer/s 1.18e+11 | flops 6.82e+10 | intensity 21.1667 | teraflops/s 2.4887
n=32768 | dim=128 | average time (ms): 5.00 | hbm xfer/s 6.45e+11 | flops 1.37e+11 | intensity 42.5000 | teraflops/s 27.4045
n=8192 | dim=256 | average time (ms): 5.01 | hbm xfer/s 6.43e+11 | flops 2.74e+11 | intensity 85.1667 | teraflops/s 54.7893
n=2048 | dim=512 | average time (ms): 5.98 | hbm xfer/s 5.39e+11 | flops 5.49e+11 | intensity 170.5000 | teraflops/s 91.8472
n=512 | dim=1024 | average time (ms): 6.96 | hbm xfer/s 4.63e+11 | flops 1.10e+12 | intensity 341.1667 | teraflops/s 157.9009
n=128 | dim=2048 | average time (ms): 12.98 | hbm xfer/s 2.48e+11 | flops 2.20e+12 | intensity 682.5000 | teraflops/s 169.4245
n=32 | dim=4096 | average time (ms): 24.50 | hbm xfer/s 1.31e+11 | flops 4.40e+12 | intensity 1,365.1667 | teraflops/s 179.5144
n=8 | dim=8192 | average time (ms): 47.56 | hbm xfer/s 6.77e+10 | flops 8.80e+12 | intensity 2,7